In [1]:
### '''Import packages'''

#import schedule
import time
import numpy as np
import pandas as pd
import urllib
import urllib.request
import requests
import re
from bs4 import BeautifulSoup
from html.parser import HTMLParser
from datetime import datetime

###################################
'''Open URL and Beautiful Soup '''

urldata = "https://www.basketball-reference.com/leagues/NBA_2023_games-february.html" 
#urldata = "https://www.basketball-reference.com/leagues/NBA_2023_totals.html" 
#urldata = "https://www.basketball-reference.com/leagues/NBA_2018_per_game.html"
#urldata = "https://www.basketball-reference.com/leagues/NBA_2021_per_game.html"
r = requests.get(urldata)
soup_new = BeautifulSoup(r.text,'html.parser')
league_table = soup_new.find('table', class_='suppress_glossary sortable stats_table')

###################################
'''Defined functions below.'''

def Source_check():
    global urldata
    a = urllib.request.urlopen(urldata)
    if a.getcode() == 200:
        print("Result code: " + str(a.getcode()))
    else:
        print("Error, cannot parse results")

def Table_name():
    Title = soup_new.find('title')
    Title = re.search('<title>(.*?)</title>', str(Title)).group(1)
    print(Title)

def url():
    for link in soup_new.find_all('a'):
        print(link.get('href'))

def Header_row():
	head = league_table.find('thead')
	col = []
	column_name = re.findall('label="(.+?)"', str(head))
	st = column_name
	for i in st:
		list_i = [i]
		col.extend(list_i)
	#column_name.remove('" class=')	    
	return(column_name)

def nba_player_data():
	i = 0
	nba_player_data_list = []
	for team in league_table.find_all('tbody'):
		rows = team.find_all('tr')
		for row in rows:
			if i < 2000:
				i += 1
				nba_players = row.find_all('th')+ row.find_all('td')
				for player in nba_players:
					nba_player_data_list.append(player)
	return(nba_player_data_list)

def actual_data():
	list_ = nba_player_data()
	max_list_count = len(list_)
	col = []
	for num in range (0,max_list_count,1):
		#if num ==0 or num%10==0:
		#	name = re.search('fcgi?"(.+?)"',str(list_[num]))
		#	if name:
		#		col.append(name.group(1))
		#else:
			actual_value = re.search('">(.+?)<',str(list_[num]))
			if actual_value is not None:
				col.append(actual_value.group(1))
			else:
				col.append('0.0')
	return(col)

def list_partition():
	main = actual_data()
	start= 0
	int = 11
	variable_assign = ['x']
	tot = len(main)
	df = pd.DataFrame(columns = Header_row())
	while start < tot:
		for part in variable_assign:
			part = main[start:(start + int)]
			df_length = len(df)
			df.loc[df_length] = part
			break
		start += int
	return(df)
###################################
'''Test functions'''

#Source_check()
#Table_name()
#Header_row()
#nba_player_data()
#actual_data()
#list_partition()
#print(len(actual_data()))

###################################
'''File Name Path Details'''

now = datetime.now()
file_date = (now.strftime("%d_%b_%Y"))
file_date.replace(r"/",".")
#file_name = "NBA_Player_Stats2.csv"
file_name = "NBAGameResults_" + str(file_date) + '.csv'
###################################
'''Dataframe (pandas)'''

df = list_partition()  
df['Visitor/Neutral'] = df['Visitor/Neutral'].str[16:19]
df['Home/Neutral'] = df['Home/Neutral'].str[16:19]
df['Date'] = df['Date'].str[-12:]
df['Date'] = df['Date'].astype('datetime64[ns]')
#df['Date'] = df['Date'].dt.strftime('%d,%m,%Y')
df.columns = ['Date', 'Start_Time_ET', 'Visitor', 'Visitor_Points', 'Home', 'Home_Points', 'Box_Score', 'OT_Flag', 'Attendance', 'Arena', 'Notes']
df.loc[df["OT_Flag"] == '0.0', "OT_Flag"] = ' '
df = df.drop(df.columns[[6,8,9,10]],axis = 1)
#df["Timestamp"] = datetime.datetime.now()
df = df.reset_index(drop=True)
df = df.rename_axis('Index')
#print(dh)
#print(df)
df.to_csv(file_name)

###################################